## API OpenAQ

Este archivo contiene los pasos realizados para la obtención de datos de calidad de aire a través de la API OpenAQ.

In [1]:
import json
import requests
import os
import pymongo

#### Conexión con MongoDB para subir los datos extraidos de la API

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ETL_project"]
collection = db["airq"]

#### Conexión con la API

In [3]:
github = os.getenv("aq_pass")
with open('../aq_pass.txt', 'r') as file:
    token = file.read()
HEADER = {'Authorization': f"password {token}"}

### List locations in Spain that measure PM2.5

In [59]:
country_code = "ES" # Españita

parameter = "pm25" # can also use the parameter id, 2  

url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.content)

b'{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":253},"results":[{"id":278343,"city":null,"name":"Calle Azafran","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":130,"unit":"particles/cm\xc2\xb3","count":201339,"average":0.03981342938778135,"lastValue":0.01,"parameter":"um025","displayName":"um025 particles/cm\xc2\xb3","lastUpdated":"2023-11-05T12:44:01+00:00","parameterId":130,"firstUpdated":"2022-01-24T07:59:57+00:00","manufacturers":null},{"id":19,"unit":"\xc2\xb5g/m\xc2\xb3","count":201339,"average":4.137699239873334,"lastValue":0.0,"parameter":"pm1","displayName":"pm1 \xc2\xb5g/m\xc2\xb3","lastUpdated":"2023-11-05T12:44:01+00:00","parameterId":19,"firstUpdated":"2022-01-24T07:59:57+00:00","manufacturers":null},{"id":126,"unit":"particles/cm\xc2\xb3","count":201339,"average":0.4212808508286896,"lastValue":0.08,"parameter":"um010","displayName":"um010 particles/cm\xc2\xb3","lastUpdated":"2023-11-

In [5]:
data_dict = response.json()

In [6]:
data = data_dict['results']  # Extraer la lista de documentos

In [7]:
# Inserta los datos en la colección
collection.insert_many(data)

## Locations by country

In [8]:
country_code = "ES"

url = f"https://api.openaq.org/v2/locations?country={country_code}"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

#print(response.content)

In [9]:
data2 = response.content

In [10]:
data_dict2 = json.loads(data2)

In [11]:
data2 = data_dict2['results']

In [12]:
collection.insert_many(data2)

In [58]:
import requests

url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&coordinates=41.38879%2C2.15899&radius=1000&order_by=lastUpdated&dump_raw=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":2},"results":[{"id":3298,"city":"Barcelona","name":"Barcelona","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":4,"unit":"µg/m³","count":3602,"average":283.9267548321465,"lastValue":200.0,"parameter":"co","displayName":"co µg/m³","lastUpdated":"2023-11-05T03:00:00+00:00","parameterId":4,"firstUpdated":"2023-05-06T02:00:00+00:00","manufacturers":null},{"id":2,"unit":"µg/m³","count":761,"average":9.736842105263158,"lastValue":4.0,"parameter":"pm25","displayName":"pm25 µg/m³","lastUpdated":"2023-11-05T03:00:00+00:00","parameterId":2,"firstUpdated":"2023-05-07T02:00:00+00:00","manufacturers":null},{"id":1,"unit":"µg/m³","count":753,"average":16.055555555555557,"lastValue":13.0,"parameter":"pm10","displayName":"pm10 µg/m³","lastUpdated":"2023-11-05T03:00:00+00:00","parameterId":1,"firstUpdated":"2023-05-07T02:00:00+00:00","manufacturers":null},{"id":6,"unit"

### All id in Madrid

In [66]:
url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&radius=1000&city=Madrid&order_by=lastUpdated&dump_raw=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

#print(response.text)

In [67]:
data_mad = response.content
data_dict_id_mad = json.loads(data_mad)
data_id_mad = data_dict_id_mad['results']

In [68]:
collection.insert_many(data_id_mad)

### Average values Madrid 2022

In [32]:
#id: 4318, "city" = Madrid, "name" = "ES1422A"

url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&coordinates=40.4165%2C-3.70256&radius=1000&order_by=lastUpdated&dump_raw=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":1},"results":[{"id":4318,"city":"Madrid","name":"ES1422A","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":19843,"unit":"µg/m³","count":3323,"average":6.306473829201102,"lastValue":2.0,"parameter":"no","displayName":"no µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":19843,"firstUpdated":"2023-03-29T13:00:00+00:00","manufacturers":null},{"id":5,"unit":"µg/m³","count":22916,"average":31.321562663371513,"lastValue":40.0,"parameter":"no2","displayName":"no2 µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":5,"firstUpdated":"2016-11-18T02:00:00+00:00","manufacturers":null},{"id":6,"unit":"µg/m³","count":22659,"average":4.864604877110288,"lastValue":2.0,"parameter":"so2","displayName":"so2 µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":6,"firstUpdated":"2016-11-18T00:00:00+00:00","manufacturers":null},{"id":4,"unit

In [15]:
data3 = response.content
data_dict3 = json.loads(data3)
data3 = data_dict3['results']

In [16]:
collection.insert_many(data3)

### Monthly PM2.5 Barcelona 2022

In [17]:
#id: 276005, "name" = "Starlab Barcelona"

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=276005&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [18]:
data4 = response.content
data_dict4 = json.loads(data4)
data4 = data_dict4['results']

In [19]:
collection.insert_many(data4)

### Monthly PM2.5 Sevilla 2022

In [20]:
#id: 4219, "name" = "ES0817A"

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=4219&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [21]:
data5 = response.content
data_dict5 = json.loads(data5)
data5 = data_dict5['results']

In [22]:
collection.insert_many(data5)

### Monthly PM2.5 Valencia 2022

In [34]:
#id: 4316 "name" = ""ES1181A" 

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=4316&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [35]:
data6 = response.content
data_dict6 = json.loads(data6)
data6 = data_dict6['results']

In [36]:
collection.insert_many(data6)

### Monthly PM2.5 Bilbao 2022

In [26]:
#id: 3264, "city" = "Araba/Álava", "name" = "ES1489A"

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=406370&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [27]:
data7 = response.content
data_dict7 = json.loads(data7)
data7 = data_dict7['results']

In [81]:
data7

[{'id': 406370,
  'name': 'CEIP Cervantes HLHI',
  'hour': None,
  'day': None,
  'month': '2022-12-01',
  'year': None,
  'hod': None,
  'dow': None,
  'average': 8.107481763020102,
  'measurement_count': 423,
  'parameter': 'pm25',
  'parameterId': 2,
  'displayName': 'PM2.5',
  'unit': 'µg/m³',
  'first_datetime': '2022-12-01T19:00:00Z',
  'last_datetime': '2022-12-31T00:00:00Z',
  '_id': ObjectId('6547781f77c26bb0875ea1d9')}]

In [28]:
collection.insert_many(data7)

### Monthly PM2.5 Málaga 2022

In [29]:
#id:69693, "name": "El Candado"

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=69693&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [77]:
data8 = response.content
data_dict8 = json.loads(data8)
data_dict8

{'meta': {'name': 'openaq-api',
  'license': '',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 0},
 'results': []}

In [31]:
collection.insert_many(data8)

### Monthly PM2.5 Madrid 2022

In [62]:
#id:3265, "name": "ES1890A", city: "Madrid"

url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=3288&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [87]:
data9 = response.content
data_dict9 = json.loads(data9)
data9 = data_dict9['results']

In [53]:
collection.insert_many(data9)

In [86]:
import requests
import pymongo
import json
# Lista de IDs de ubicación
location_ids_mad = [3265,3364,3367,4244,4265,4267,4270,4274,4275,4280,4283,4285,4244,4241,4342,4341,4338,4333,4332,4331,4329,4328,
 4327,4323,4320,3265,4317,4314,4313,4312,3288,4318,4342,3265,3288,3295,3353,3364,3367,4241,4244,4247,4248,4249,4255,4258,4264,4265,
 4267,4270,4271,4274,4275,4280,4281,4283,4285,4287,4288,4296,4298,4305,4306,4312,4313,4314,4317,3223,4320,4323,4327,4328,4329,4331,4332,4333,4338,4341,4318]
headers = {"accept": "application/json"}

# Conectarse a la base de datos MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ETL_project"]
collection = db["airq"]

for location_id in location_ids_mad:
    # Construye la URL con el ID de ubicación actual
    url = f"https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id={location_id}&spatial=location&limit=100&page=1"
    # Realiza la solicitud GET para la URL actual
    response = requests.get(url, headers=headers)
    try:
        # Procesa la respuesta y almacena los datos en MongoDB como un objeto separado
        if response.status_code == 200:
            data = response.content
            data_dict = json.loads(data)
            data = data_dict.get('results', [])  # Obtiene la parte 'results' o una lista vacía si no existe
            if data:
                collection.insert_many(data)
                print(f"Datos para la ubicación {location_id} insertados en MongoDB.")
            else:
                print(f"Datos para la ubicación {location_id} están vacíos.")
        else:
            print(f"Error al obtener datos para la ubicación {location_id}. Código de estado: {response.status_code}")
    except TypeError as e:
        print(f"Error al insertar datos para la ubicación {location_id}: {e}")

Datos para la ubicación 3265 insertados en MongoDB.
Datos para la ubicación 3364 insertados en MongoDB.
Datos para la ubicación 3367 insertados en MongoDB.
Datos para la ubicación 4244 insertados en MongoDB.
Datos para la ubicación 4265 insertados en MongoDB.
Datos para la ubicación 4267 insertados en MongoDB.
Datos para la ubicación 4270 insertados en MongoDB.
Datos para la ubicación 4274 insertados en MongoDB.
Datos para la ubicación 4275 insertados en MongoDB.
Datos para la ubicación 4280 insertados en MongoDB.
Datos para la ubicación 4283 insertados en MongoDB.
Datos para la ubicación 4285 insertados en MongoDB.
Datos para la ubicación 4244 insertados en MongoDB.
Datos para la ubicación 4241 están vacíos.
Datos para la ubicación 4342 están vacíos.
Datos para la ubicación 4341 están vacíos.
Datos para la ubicación 4338 insertados en MongoDB.
Datos para la ubicación 4333 están vacíos.
Datos para la ubicación 4332 están vacíos.
Datos para la ubicación 4331 están vacíos.
Datos para la 